In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


# 1. Import

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.options.display.max_columns = 999

In [4]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv")

In [5]:
sub.head()

,id,target
0,5,0.5
1,6,0.5
2,8,0.5
3,9,0.5
4,11,0.5


In [6]:
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,5,A,F,A,A,F,BI,A,AH,AX,A,LF,A,A,A,A,B,D,D,B,0.708120,0.735690,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489
1,6,A,H,C,A,E,AB,I,F,N,A,MC,A,A,A,B,D,B,D,B,0.611637,0.313703,0.928885,0.516602,0.600169,0.795224,0.248987,0.654614,0.347944,0.565520,0.388580
2,8,A,N,C,A,F,AB,A,AH,BC,A,FN,A,A,A,B,B,D,D,B,0.456289,0.448201,0.424876,0.344729,0.242073,0.270632,0.746740,0.335590,0.341238,0.252289,0.411592
3,9,B,L,C,A,F,BI,A,E,AX,A,HK,A,A,A,A,B,D,B,B,0.338692,0.666092,0.598943,0.561971,0.806347,0.735983,0.538724,0.381566,0.481660,0.348514,0.325723
4,11,A,F,A,B,F,BI,A,AH,I,A,HX,A,A,A,A,D,D,D,B,0.631671,0.772229,0.479572,0.767745,0.252454,0.354810,0.178920,0.763479,0.562491,0.466261,0.585781


In [7]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,A,LO,A,A,A,A,B,D,D,B,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,F,HJ,A,B,A,B,D,B,D,B,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,L,DJ,A,B,A,A,B,D,D,B,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,F,KV,A,A,A,A,B,D,D,B,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,A,DP,A,A,A,B,B,B,D,B,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


# 2. Preprocessing
### Score : ROC_CURVE

In [8]:
allData = pd.concat([train, test])
allData

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,A,LO,A,A,A,A,B,D,D,B,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0.0
1,1,A,I,A,A,E,BI,K,W,AD,F,HJ,A,B,A,B,D,B,D,B,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0.0
2,2,A,K,A,A,E,BI,A,E,BM,L,DJ,A,B,A,A,B,D,D,B,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0.0
3,3,A,K,A,C,E,BI,A,Y,AD,F,KV,A,A,A,A,B,D,D,B,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0.0
4,4,A,I,G,B,E,BI,C,G,Q,A,DP,A,A,A,B,B,B,D,B,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,499983,A,N,A,D,F,BI,A,AF,AX,A,DP,A,A,A,B,D,B,C,B,0.451496,0.361426,0.351946,0.327670,0.205547,0.679195,0.485967,0.319130,0.520681,0.519545,0.427119,NaN
199996,499984,B,I,A,B,E,BI,A,AS,S,E,IE,A,A,A,B,B,D,D,B,0.862988,0.551106,0.628843,0.677765,0.624935,0.555306,0.242424,0.700829,0.531728,0.528427,0.922645,NaN
199997,499987,A,L,D,A,H,BI,A,AN,J,A,HV,A,A,A,B,D,B,D,B,0.463067,0.812384,0.803348,0.324762,0.665624,0.488447,0.853213,0.578641,0.811941,0.537106,0.531758,NaN
199998,499994,A,K,A,C,F,BI,A,R,BM,A,HK,A,A,A,A,B,D,D,B,0.519671,0.811282,0.820635,0.561449,0.797434,0.555089,0.746532,0.369986,0.438712,0.715524,0.381978,NaN


In [9]:
categories = [col for col in allData.columns[1:] if allData[col].dtypes == 'object']
num_features = [col for col in allData.columns[1:] if allData[col].dtypes != 'object']

In [10]:
le = LabelEncoder()
for col in allData.columns[allData.dtypes == 'object']:
    allData[col] = le.fit_transform(list(allData[col]))
allData.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,0,8,0,1,1,33,0,44,54,0,266,0,0,0,0,1,3,3,1,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0.0
1,1,0,8,0,0,4,33,8,48,3,5,167,0,1,0,1,3,1,3,1,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0.0
2,2,0,10,0,0,4,33,0,30,38,9,73,0,1,0,0,1,3,3,1,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0.0
3,3,0,10,0,2,4,33,0,50,3,5,249,0,0,0,0,1,3,3,1,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0.0
4,4,0,8,6,1,4,33,2,32,54,0,79,0,0,0,1,1,1,3,1,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1.0


In [11]:
allData2 = allData.drop(['id', 'target'],1)
label = train['target']

In [12]:
train2 = allData2[:len(train)]
test2 = allData2[len(train):]
train2.shape, test2.shape

((300000, 30), (200000, 30))

# 3. Modeling : StratifyKFold and Ensemble

In [13]:
# models with training

def model_catboost(X_train, y_train, X_valid, y_valid, test2):
    cat = CatBoostClassifier(task_type='GPU')
    cat.fit(X_train, y_train, verbose=200, cat_features = categories)
    valid_pred = cat.predict_proba(X_valid)[:,1]
    test_pred = cat.predict_proba(test2)[:,1]
    score_pred = roc_auc_score(y_valid, valid_pred)
    return score_pred, test_pred

def model_lgbm(X_train, y_train, X_valid, y_valid, test2):
    lgb = LGBMClassifier()
    lgb.fit(X_train, y_train)
    valid_pred = lgb.predict_proba(X_valid)[:,1]
    test_pred = lgb.predict_proba(test2)[:,1]
    score_pred = roc_auc_score(y_valid, valid_pred)
    return score_pred, test_pred

def model_xgb(X_train, y_train, X_valid, y_valid, test2):
    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)
    valid_pred = xgb.predict_proba(X_valid)[:,1]
    test_pred = xgb.predict_proba(test2)[:,1]
    score_pred = roc_auc_score(y_valid, valid_pred)
    return score_pred, test_pred

## 3-1. StratifyKFold

In [14]:
%%time
# StratifyKFold

n_counts = 5

skf = StratifiedKFold(n_splits = n_counts, shuffle = True, random_state=42)

result = {}
scores = {}

model_list = ['cat', 'lgb', 'xgb']
for m in model_list:
    result[m] = 0
    scores[m] = []

i = 1
for train_index, valid_index in skf.split(train2, label):
    X_train, y_train = train2.loc[train_index], label[train_index]
    X_valid, y_valid = train2.loc[valid_index], label[valid_index]
    
    print(f"\n skf times # {i}\n")
    for m in model_list:
        if m == 'cat':
            score_pred, test_pred = model_catboost(X_train, y_train, X_valid, y_valid, test2)
        elif m == 'lgb':
            score_pred, test_pred = model_lgbm(X_train, y_train, X_valid, y_valid, test2)
        elif m == 'xgb':
            score_pred, test_pred = model_xgb(X_train, y_train, X_valid, y_valid, test2)
        scores[m].append(score_pred)
        result[m] += test_pred/n_counts 
        print(f"times # {i} {m} train and inference complete")
        
    i += 1
    
print('\n####################### DONE ###############################')


 skf times # 1

Learning rate set to 0.024938
0:	learn: 0.6799831	total: 123ms	remaining: 2m 2s
200:	learn: 0.3708748	total: 8.97s	remaining: 35.7s
400:	learn: 0.3602911	total: 17.3s	remaining: 25.8s
600:	learn: 0.3560058	total: 25.4s	remaining: 16.9s
800:	learn: 0.3531746	total: 33.7s	remaining: 8.37s
999:	learn: 0.3510054	total: 42.4s	remaining: 0us
times # 1 cat train and inference complete
times # 1 lgb train and inference complete
[13:48:08] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
times # 1 xgb train and inference complete

 skf times # 2

Learning rate set to 0.024938
200:	learn: 0.3710006	total: 8.36s	remaining: 33.3s
400:	learn: 0.3607459	total: 16.8s	remaining: 25.1s
600:	learn: 0.3562100	total: 26s	remaining: 17.2s
800:	learn: 0.3534532	total: 33.9s	remaining: 8.41s
999:	learn:

## 3-2. Ensemble

In [15]:
for key, score in scores.items():
    print(f"{key} : {np.mean(score)}")

cat : 0.8878027485671318
lgb : 0.8886448816859309
xgb : 0.8888603236087567


In [16]:
weights = [0.3, 0.35, 0.35]
final_score = 0
for k, w in zip(result.keys(), weights):
    final_score += result[k] * w
    print(f"{k} result : {result[k]}")
print(f"final_score : {final_score}")

cat result : [0.14454085 0.56059473 0.03761378 ... 0.71916002 0.10511554 0.48863405]
lgb result : [0.12034276 0.55146637 0.04433476 ... 0.70746494 0.16071512 0.3247914 ]
xgb result : [0.10539148 0.5227498  0.02953003 ... 0.65384614 0.11762388 0.42457357]
final_score : [0.12236924 0.54415407 0.03713681 ... 0.69220688 0.12895331 0.40886795]


# 4. Submission

In [17]:
sub['target'] = final_score
sub.to_csv("tabular_sub.csv", index=0)
sub

,id,target
0,5,0.112867
1,6,0.537108
2,8,0.036932
3,9,0.160307
4,11,0.143236
...,...,...
199995,499983,0.868582
199996,499984,0.035543
199997,499987,0.680656
199998,499994,0.139169
